In [2]:
import pandas as pd
import os
import re
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.set_option("display.max_rows", 2000)

In [44]:
trans_key_cols = ["date", "original_description", "amount"]

def to_snake(txt):
    words = txt.lower().split(" ")
    return "_".join(words)
    

def get_trans_keys(df):
    return df[trans_key_cols].drop_duplicates()    
    

def get_net_amount(row):
    if row['transaction_type'] == 'debit':
        return row['amount']
    else:
        return row['amount'] * -1

    
def load_transactions(path):
    df = pd.read_csv(path)
    df.columns = [to_snake(col) for col in df.columns]
    df["amount"] = df.apply(get_net_amount, axis = 1)
    df.rename(columns = {"description": "mint_description"}, inplace = True)
    return df.drop(["transaction_type","category","labels","notes"], axis = 1)
    
    
def get_uncatted_trans():
    """returns a data frame of transactions without a category"""
    
    raw_trans = load_transactions("./data/raw/transactions.csv")
    
    # if catted_trans exist, get difference
    if "transactions.csv" in os.listdir("./data/categorized/"):
        catted = load_transactions("./data/categorized/transactions.csv")
        catted_keys = get_trans_keys(catted)
        catted_keys.columns = ["catted_" + col for col in catted_keys.columns]
        join_catted = raw_trans.merge(catted_keys, left_on = trans_key_cols, right_on = catted_keys.columns, how = "left")
        uncatted_trans = join_catted[catted_keys.columns[0].isna()].drop(catted_keys.columns)
        
    else:
        return raw_trans
    

In [45]:
get_uncatted_trans()

,date,mint_description,original_description,amount,account_name
0,6/19/2021,AUTH : ORIGIN COFFEE LAB & KITCH,AUTH : ORIGIN COFFEE LAB & KITCH,3.02,Fidelity Rewards Visa Signature
1,6/18/2021,Amazon,AMZN Mktp US,7.41,CREDIT CARD
2,6/18/2021,AUTH : TST* NORTHSIDE SOCIAL COF,AUTH : TST* NORTHSIDE SOCIAL COF,17.88,Fidelity Rewards Visa Signature
3,6/18/2021,Amazon,AMZN Mktp US,52.99,CREDIT CARD
4,6/18/2021,Prime Video,Prime Video,3.99,CREDIT CARD
5,6/17/2021,Amazon,AMZN Mktp US,31.79,CREDIT CARD
6,8/14/2018,Amazon,POS DEBIT AMZN Mktp US ...,19.02,CHASE COLLEGE
7,8/14/2018,Amazon,POS DEBIT AMZN Mktp US ...,245.40,CHASE COLLEGE
8,6/18/2021,Gold's Gym,GOLDS GYM VA SOUTH AR 703-683-46,24.99,CHASE COLLEGE
9,6/18/2021,FID BKG SVC LLC MONEYLINE PPD ID: 1035141383,FID BKG SVC LLC MONEYLINE,1000.00,CHASE COLLEGE


In [37]:
df.head()

,date,description,original_description,amount,transaction_type,category,account_name,labels,notes
0,6/19/2021,AUTH : ORIGIN COFFEE LAB & KITCH,AUTH : ORIGIN COFFEE LAB & KITCH,3.02,debit,Coffee Shops,Fidelity Rewards Visa Signature,NaN,NaN
1,6/18/2021,Amazon,AMZN Mktp US,7.41,debit,Shopping,CREDIT CARD,NaN,NaN
2,6/18/2021,AUTH : TST* NORTHSIDE SOCIAL COF,AUTH : TST* NORTHSIDE SOCIAL COF,17.88,debit,Restaurants,Fidelity Rewards Visa Signature,NaN,NaN
3,6/18/2021,Amazon,AMZN Mktp US,52.99,debit,Shopping,CREDIT CARD,NaN,NaN
4,6/18/2021,Prime Video,Prime Video,3.99,debit,Movies & DVDs,CREDIT CARD,NaN,NaN


In [36]:
df = pd.read_csv('./data/raw/transactions.csv')
df.columns = [to_snake(col) for col in df.columns]

In [7]:
def to_snake(txt):
    words = txt.lower().split(" ")
    return "_".join(words)

In [8]:
df.columns = [to_snake(col) for col in df.columns]

In [9]:
df.head()

,date,description,original_description,amount,transaction_type,category,account_name,labels,notes
0,6/19/2021,AUTH : ORIGIN COFFEE LAB & KITCH,AUTH : ORIGIN COFFEE LAB & KITCH,3.02,debit,Coffee Shops,Fidelity Rewards Visa Signature,NaN,NaN
1,6/18/2021,Amazon,AMZN Mktp US,7.41,debit,Shopping,CREDIT CARD,NaN,NaN
2,6/18/2021,AUTH : TST* NORTHSIDE SOCIAL COF,AUTH : TST* NORTHSIDE SOCIAL COF,17.88,debit,Restaurants,Fidelity Rewards Visa Signature,NaN,NaN
3,6/18/2021,Amazon,AMZN Mktp US,52.99,debit,Shopping,CREDIT CARD,NaN,NaN
4,6/18/2021,Prime Video,Prime Video,3.99,debit,Movies & DVDs,CREDIT CARD,NaN,NaN


In [18]:
df[['date','original_description','amount']].value_counts().sort_values(ascending=False)

date        original_description                                                               amount 
2/07/2020   18TH STREET LOUNGE WASHINGTON DC                                                   9.00       3
3/02/2020   DECADES NORTHWEST WAS DC                                                           25.00      2
3/10/2021   NON-CHASE ATM FEE-WITH                                                             5.00       2
10/15/2019  DD/BR #335956 Q35 FALLS CHURCH V                                                   2.39       2
12/05/2019  HSA BANK         OTHER TRAN                                                        0.02       2
11/16/2020  SQ *THE VEIL BREWING CO Richmond                                                   8.94       2
6/29/2018   7-ELEVEN ARLINGTON VA06/29                                                         2.35       2
6/01/2021   SQ *BREAD & WATER COMPA Arlingto                                                   4.85       2
8/06/2018   WHITLOWS ON WILSON AR

In [439]:
credits = df[df['transaction_type'] == 'credit']
debits = df[df['transaction_type'] == 'debit']

In [440]:
def edit_descr(descr, descr_map):
    for key in descr_map.keys():
        if  key.lower() in descr.lower():
            return descr_map[key]
    return descr

In [441]:
# map for credit descriptions. If description contains key, change description to value
descr_map = {
    'ironnet': 'Ironnet Cybersecuirty',
    'robinhood': 'Robinhood',
    'home depot': 'Home Depot',
    'amzn': 'Amazon',
    'amazon': 'Amazon',
    'venmo': 'Venmo',
    'cromwell': 'crowmell zelle',
    'zelle': 'zelle',
    'remote online deposit': 'remote online deposit',
    'simple': 'Simple Bank',
    'atm': 'ATM',
    'education student ln': 'student loans',
    'coinbase': 'Coinbase',
    'chase card': 'chase card',
    'citi': 'Citi Card',
    'usaa': 'USAA',
    'VHC': 'VHC',
    'harris': 'Harris Teeter',
    'parking': 'parking fee',
    'garag': 'parking fee',
    'parkmobile': 'parking fee',
    'long term': 'parking fee',
    '5guys': '5 Guys',
    'prime video': 'prime video',
    'cubesmart': 'CubeSmart',
    'hbo': 'HBO',
    'bird app': 'Bird',
    'bund up': 'Bund Up',
    'total wine': 'Total Wine',
    'bread & water': 'Bread & Water',
    'foreign exchange rate adjustment fee': 'foreign exchange fee',
    'giant': 'Giant Groceries',
    'idego': 'Idego',
    'goat': 'GOAT',
    'lim*ride': 'lim*ride',
    'onlyfans': 'onlyfans',
    'postmates': 'postmates',
    'philz coffee': 'philz coffee',
}

In [442]:
def regex_filter(descr):
    patterns = [
        # only digits word
        '\s\d+',
        
        # state and forward slash
        '\s[A-Z]{1,2}/[0-9]{2}',
        
        # filter LLC extension
        ', LLC.*',
        'LLC.*',
        
        # filter website extension
        '\sWWW.*',
        '\s\S\.com',
        
        # simple filters
        '\sWEB\s',
        'ID:',
        
        'TST\*\s',
        
        '\sHTTP.*',
    ]
    
    new_descr = descr
    for pattern in patterns:
        new_descr = re.sub(pattern, '', new_descr)
        
    return new_descr

In [444]:
credits['edited_description'] = credits['description'].apply(lambda x: regex_filter(edit_descr(x, descr_map)))
credits[['original_description','description','edited_description']]

,original_description,description,edited_description
5,CITICARDS CASH REWARD,Citicards Cash Reward,Citi Card
8,Cash Redemption,Cash Redemption,Cash Redemption
35,IRONNET CYBERSEC DIRECT DEP,IRONNET CYBERSEC DIRECT DEP PPD ID: 9111111103,Ironnet Cybersecuirty
40,ROBINHOOD CREDITS,ROBINHOOD CREDITS PPD ID: 5321710001,Robinhood
42,ROBINHOOD CREDITS,ROBINHOOD CREDITS PPD ID: 5321710001,Robinhood
55,ROBINHOOD CREDITS,ROBINHOOD CREDITS PPD ID: 5321710001,Robinhood
75,IRONNET CYBERSEC DIRECT DEP,IRONNET CYBERSEC DIRECT DEP PPD ID: 9111111103,Ironnet Cybersecuirty
92,VENMO CASHOUT,Venmo Cashout Ppd,Venmo
122,IRONNET CYBERSEC DIRECT DEP,IRONNET CYBERSEC DIRECT DEP PPD ID: 9111111103,Ironnet Cybersecuirty
151,VENMO CASHOUT,Venmo Cashout Ppd,Venmo


In [435]:
df['edited_description'] = df['description'].apply(lambda x: regex_filter(edit_descr(x, descr_map)))
df[['original_description','description','edited_description']].nunique()

original_description    893
description             766
edited_description      459
dtype: int64

In [436]:
# map of main categories and list of sub categories
categories = {
    "eating/drinking out": ['delivery', 'resturant', 'coffe_shops', "bars"],
    "home": ["rent", "utilities", "home items"],
    "grocery store": [],
    "entertainment media": ["movies", "music", "news", "books"],
    "amusement": [],
    "student loans": [],
    "Transportation": ["uber","public"],
    "lodging": ["airbnb", "hotel"],
    "Shopping": ["music", "sports", "clothing", "electronics"],
    "health": ["doctor", "pharmacy", "hygene", "gym", "therapy"],
    "travel": ["airplane"],
    "government": ["taxes", "dmv"],
    "insurance": ["renters", "auto"]
}

In [437]:
df[['edited_description', 'category']].sort_values('edited_description').drop_duplicates()

,edited_description,category
1194,&PIZZA - BALLSTON ARLINGTON,Restaurants
957,0031 MCLEAN MCLEAN,Clothing
908,18th Street Lounge,Food & Dining
1045,1WSHGTN NTNLS WASHINGTO N,Business Services
256,2097-CLT TRIP ADVISOR CHARLOTTE,Air Travel
1021,2Checkout,Electronics & Software
218,3 Idts Incorporated,Gas & Fuel
1254,5 Guys,Fast Food
925,618 COCKTAIL AND WHISKE WASHINGT ON,Alcohol & Bars
1111,7-Eleven,Fast Food


In [448]:
def get_signed_amount(row):
    if row['transaction_type'] == 'debit':
        return row['amount']
    else:
        return row['amount'] * -1

df['signed_amount'] = df.apply(get_signed_amount, axis = 1)

In [451]:
# net gains from robinhood
df.loc[df['edited_description'] == 'Robinhood', 'signed_amount'].sum()

-194.62000000000023